In [1]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

W i n d o w s   S u b s y s t e m   f o r   L i n u x   h a s   n o   i n s t a l l e d   d i s t r i b u t i o n s . 
 
 
 D i s t r i b u t i o n s   c a n   b e   i n s t a l l e d   b y   v i s i t i n g   t h e   M i c r o s o f t   S t o r e : 
 
 
 h t t p s : / / a k a . m s / w s l s t o r e 
 
 
 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

### OpenAI Gym

We're gonna spend several next weeks learning algorithms that solve decision processes. We are then in need of some interesting decision problems to test our algorithms.

That's where OpenAI Gym comes into play. It's a Python library that wraps many classical decision problems including robot control, videogames and board games.

So here's how it works:

In [3]:
import gym

env = gym.make("MountainCar-v0")
env.reset()
env.render('rgb_array')
# plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Action space: Discrete(3)


C:\Users\adhocmaster\anaconda3\envs\gymnes\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Note: if you're running this on your local machine, you'll see a window pop up with the image above. Don't close it, just alt-tab away.

### Gym interface

The three main methods of an environment are
* `reset()`: reset environment to the initial state, _return first observation_
* `render()`: show current environment state (a more colorful version :) )
* `step(a)`: commit action `a` and return `(new_observation, reward, is_done, info)`
 * `new_observation`: an observation right after committing the action `a`
 * `reward`: a number representing your reward for committing action `a`
 * `is_done`: True if the MDP has just finished, False if still in progress
 * `info`: some auxiliary stuff about what just happened. For now, ignore it.

In [4]:
obs0 = env.reset()
env.render()
print("initial observation code:", obs0)

# Note: in MountainCar, observation is just two numbers: car position and velocity

initial observation code: [-0.57424307  0.        ]


In [5]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
env.render()

print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)

# Note: as you can see, the car has moved to the right slightly (around 0.0005)

taking action 2 (right)
new observation code: [-0.5728647   0.00137837]
reward: -1.0
is game over?: False


### Play with it

Below is the code that drives the car to the right. However, if you simply use the default policy, the car will not reach the flag at the far right due to gravity.

__Your task__ is to fix it. Find a strategy that reaches the flag. 

You are not required to build any sophisticated algorithms for now, and you definitely don't need to know any reinforcement learning for this. Feel free to hard-code :)

In [6]:
# from IPython import display

# Create env manually to set time limit. Please don't change this.
TIME_LIMIT = 250
# env = gym.wrappers.TimeLimit(
#     gym.envs.classic_control.MountainCarEnv(),
#     max_episode_steps=TIME_LIMIT + 1,
# )
actions = {'left': 0, 'stop': 1, 'right': 2}

In [54]:
prevPos = None
prevV = None
def policy(obs, t):
    global prevPos, prevV
    # Write the code for your policy here. You can use the observation
    # (a tuple of position and velocity), the current time step, or both,
    # if you want.
    position, velocity = obs
    
    # This is an example policy. You can try running it, but it will not work.
    # Your goal is to fix that. You don't need anything sophisticated here,
    # and you can hard-code any policy that seems to work.
    # Hint: think how you would make a swing go farther and faster.
    # return actions['right']
    if prevPos is None:
        prevPos = position
        prevV = velocity
        return np.random.choice([0, 2])
    
    # if car is going up, push up, if going down, push down.
    if velocity == 0.0:
        # print("velocity is 0")
        prevPos = position
        if prevV > 0:
            prevV = velocity
            return actions['left']
        return actions['right']
            
    
    if velocity > 0: # going up in left or right
        
        if prevPos > position: # going left
            prevV = velocity
            prevPos = position
            # print("going up-left action left")
            return actions['left']
        else:
            prevV = velocity
            prevPos = position
            # print("going up-right action right")
            return actions['right']
    
    
    if prevPos > position: 
        prevV = velocity
        prevPos = position
        # print("going down-left action left")
        return actions['left']
    else:
        prevV = velocity
        prevPos = position
        # print("going down-right action left")
        return actions['right']
    
            
        

In [55]:
# plt.figure(figsize=(4, 3))
# display.clear_output(wait=True)

obs = env.reset()
for t in range(TIME_LIMIT):
    # plt.gca().clear()
    
    action = policy(obs, t)  # Call your policy
    obs, reward, done, _ = env.step(action)  # Pass the action chosen by the policy to the environment
    
    # We don't do anything with reward here because MountainCar is a very simple environment,
    # and reward is a constant -1. Therefore, your goal is to end the episode as quickly as possible.

    # Draw game image on display.
    env.render()
    # plt.imshow(env.render('rgb_array'))
    
    # display.display(plt.gcf())
    # display.clear_output(wait=True)

    if done:
        print("Well done!")
        break
    # print(f"time step {t}")
    # print(obs)
else:
    print("Time limit exceeded. Try again.")

# display.clear_output(wait=True)

Well done!


In [56]:
assert obs[0] > 0.47
print("You solved it!")

You solved it!


In [57]:
env.close()